In [22]:
import pandas as pd
import numpy  as np
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

In [23]:
df = pd.read_csv("./dataset.csv")

id_start = int(df.loc[len(df)-1]["id"])

In [24]:
def generate_essay(topic, word_count=100):
    """
    Generate an essay on the given topic using OpenAI GPT-4o API.

    :param topic: The topic of the essay.
    :param word_count: Desired word count of the essay (approximate).
    :return: Generated essay as a string.
    """
    try:
        # Define the prompt
        prompt = (
            f"Write an essay about '{topic}'. "
            f"The essay should be approximately {word_count} words long."
        )

        # Call OpenAI API
        response = openai.ChatCompletion.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are an essay writer."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=word_count * 4 // 3,  # Estimate tokens for word count
            temperature=0.7                  # Adjust creativity level
        )

        # Extract and return the essay text
        essay = response['choices'][0]['message']['content'].strip()
        return essay

    except Exception as e:  # Catch generic exceptions
        return f"Error generating essay: {str(e)}"


In [79]:
def evaluate_essay(essay, topic, word_count=10, gpt_model="gpt-4o"):
    try:
        # Define the prompt
        prompt = (
            # f"Consider this essay '{essay}' which is written on the subject of this topic: '{topic}'. "
            # f"Evaluate the essay and return a real value in [0, 9] based on its relevance to the topic and its correctness. "
            # f"Your answer shoud be in this format: Scode = ...."
            # # f"Do not write anything more."
            # f"Explain how did you com to this score mathematically."
            
            f"You are an expert essay evaluator. Your task is to evaluate the quality of essays based on their relevance to the topic, coherence, organization, grammar, and depth of content."
            f" The scoring range is from 0.0 to 9.0, with 0.0 being the lowest quality and 9.0 being outstanding quality."
            f" Here is the topic: \"{topic}\"."
            f" Here is the essay:\n\n{essay}\n\n"
            f"Please provide a single numeric score (0.0 to 9.0) as the output."
        )

        # Call OpenAI API
        response = openai.ChatCompletion.create(
            model=gpt_model,
            messages=[
                {"role": "system", "content": "You evaluate essays based on quality and relevance to the topic."},
                {"role": "user", "content": prompt}
            ],
            # max_tokens = word_count
        )

        # Extract and return the essay text
        score = response['choices'][0]['message']['content'].strip()
        return score

    except Exception as e:  # Catch generic exceptions
        return f"Error generating score: {str(e)}"


In [80]:
topic = "metaphors in Ferdowsi's poems"

essay = generate_essay(topic, word_count=100)

essay

'Ferdowsi, the illustrious Persian poet, is renowned for his magnum opus, the Shahnameh, a monumental epic that weaves the history and mythology of Persia through rich, evocative language. Central to his poetic mastery is the use of metaphors, which animate the text with vivid imagery and profound symbolism. Ferdowsi employs metaphors to convey themes of heroism, destiny, and the eternal struggle between good and evil. For instance, he often likens heroes to majestic lions, symbolizing strength and nobility. Such metaphors not only embellish the narrative but also deepen its emotional resonance, allowing readers to explore the intricate layers of'

In [84]:
essay = "I am a blackboard. " + essay + " The girl is a bird."

In [95]:
models = ["gpt-3.5-turbo", "babbage-002", "gpt-4", "gpt-4o", "gpt-4o-mini"]

for item in models:
    print(item, ":  ", evaluate_essay(essay, topic, word_count=1000, gpt_model=item))
    print("="*80)

gpt-3.5-turbo :   I would rate this essay a 5.0. 

While the essay discusses the use of metaphors in Ferdowsi's poems and provides some examples, there are some issues with coherence and relevance to the topic. The mention of "The girl is a bird" at the end seems disconnected from the overarching discussion of Ferdowsi's metaphors. Additionally, the essay could benefit from a clearer structure and more in-depth analysis of how metaphors are used in Ferdowsi's work. Overall, the essay shows a good understanding of the topic but lacks depth and organization.
babbage-002 :   Error generating score: This is not a chat model and thus not supported in the v1/chat/completions endpoint. Did you mean to use v1/completions?
gpt-4 :   5.0
gpt-4o :   6.0
gpt-4o-mini :   5.0


In [18]:
topics = ["The Role of Technology in Modern Health Care",
          "Global Health Care Systems: A Comparative Analysis",
          "Mental Health Awareness and Support Systems",
          "The Impact of COVID-19 on Global Health Care",
          "Ethics in Health Care Decision-Making",
          "Health Care Accessibility and Disparities",
          "Preventative Health Care: Trends and Importance",
          "The Rising Costs of Health Care and Possible Solutions",
          "The Role of Big Data and Analytics in Health Care",
          "Health Care Workforce Challenges",
          ]

In [19]:
id_vec = []
essay_vec = []
for i in range(len(topics)):
    id = id_start + i + 1
    topic = topics[i]
    essay = generate_essay(topic)
    id_vec.append(id)
    essay_vec.append(essay)

In [20]:
res = pd.DataFrame()

res["id"]    = id_vec
res["title"] = topics
res["essay"] = essay_vec

res

,id,title,essay
0,1041,The Role of Technology in Modern Health Care,Technology plays a pivotal role in modern heal...
1,1042,Global Health Care Systems: A Comparative Anal...,Global health care systems vary significantly ...
2,1043,Mental Health Awareness and Support Systems,Mental health awareness and support systems ar...
3,1044,The Impact of COVID-19 on Global Health Care,The COVID-19 pandemic has profoundly impacted ...
4,1045,Ethics in Health Care Decision-Making,Ethics in health care decision-making is a cru...
5,1046,Health Care Accessibility and Disparities,Health care accessibility and disparities rema...
6,1047,Preventative Health Care: Trends and Importance,Preventative health care emphasizes the import...
7,1048,The Rising Costs of Health Care and Possible S...,The rising costs of health care present a sign...
8,1049,The Role of Big Data and Analytics in Health Care,Big Data and analytics have revolutionized hea...
9,1050,Health Care Workforce Challenges,The health care workforce faces significant ch...


In [21]:
df = pd.concat([df, res])
df.reset_index(inplace=True, drop=True)
df.to_csv("dataset.csv", index=False)